In [16]:
import os
import time
import numpy as np

os.chdir('..')
from model.model import model_load
from model.model import model_predict

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from model.cslib import fetch_ts, engineer_features

In [22]:
print(os.getcwd())


e:\


In [24]:
# data_dir = os.path.join("data","cs-train")
data_dir = r'E:\Workspaces\ws1\ai-enterprise-workflow-capstone\data\cs-train'

ts_all = fetch_ts(data_dir,clean=False)

... loading ts data from files


In [25]:
X,y,dates = engineer_features(ts_all['all'])
        
## Perform a train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)

In [28]:
param_grid_rf = {
    'rf__criterion': ['squared_error', 'absolute_error'],  # Updated criteria
    'rf__n_estimators': [10, 15, 20, 25, 50, 100]
}

time_start = time.time()
pipe_rf = Pipeline(steps=[('scaler', StandardScaler()), ('rf', RandomForestRegressor())])

grid = GridSearchCV(pipe_rf, param_grid=param_grid_rf, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

rf_mae = mean_absolute_error(y_test, y_pred)
rf_mse = mean_squared_error(y_test, y_pred)
rf_r2_score = r2_score(y_test, y_pred)

print("train time = ", time.strftime('%H:%M:%S', time.gmtime(time.time() - time_start)))
print("mae = {:.0f}".format(rf_mae))
print("mse = {:.0f}".format(rf_mse))
print("r2_score = {:.3f}".format(rf_r2_score))
print("best params =", grid.best_params_)


train time =  00:00:05
mae = 11353
mse = 281578469
r2_score = 0.957
best params = {'rf__criterion': 'squared_error', 'rf__n_estimators': 15}


In [31]:
param_grid_gb = {
    'gb__criterion': ['squared_error', 'friedman_mse'],  # Corrected values
    'gb__n_estimators': [10, 15, 20, 25, 50, 100]
}

time_start = time.time()
pipe_gb = Pipeline(steps=[('scaler', StandardScaler()), ('gb', GradientBoostingRegressor())])

grid = GridSearchCV(pipe_gb, param_grid=param_grid_gb, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

gb_mae = mean_absolute_error(y_test, y_pred)
gb_mse = mean_squared_error(y_test, y_pred)
gb_r2_score = r2_score(y_test, y_pred)

print("train time = ", time.strftime('%H:%M:%S', time.gmtime(time.time() - time_start)))
print("mae = {:.0f}".format(gb_mae))
print("mse = {:.0f}".format(gb_mse))
print("r2_score = {:.3f}".format(gb_r2_score))
print("best params =", grid.best_params_)


train time =  00:00:01
mae = 16175
mse = 452582614
r2_score = 0.931
best params = {'gb__criterion': 'friedman_mse', 'gb__n_estimators': 100}


In [33]:
param_grid_dt = {
    'dt__criterion': ['squared_error', 'absolute_error'],  # Corrected values
    'dt__max_depth': [5, 10, 20, 50],
    'dt__min_samples_leaf': [1, 2, 3, 4, 5]
}

time_start = time.time()
pipe_ts = Pipeline(steps=[('scaler', StandardScaler()), ('dt', DecisionTreeRegressor())])

# Removed the `iid` parameter
grid = GridSearchCV(pipe_ts, param_grid=param_grid_dt, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

dt_mae = mean_absolute_error(y_test, y_pred)
dt_mse = mean_squared_error(y_test, y_pred)
dt_r2_score = r2_score(y_test, y_pred)

print("train time = ", time.strftime('%H:%M:%S', time.gmtime(time.time() - time_start)))
print("mae = {:.0f}".format(dt_mae))
print("mse = {:.0f}".format(dt_mse))
print("r2_score = {:.3f}".format(dt_r2_score))
print("best params =", grid.best_params_)


train time =  00:00:00
mae = 11949
mse = 423462270
r2_score = 0.935
best params = {'dt__criterion': 'squared_error', 'dt__max_depth': 50, 'dt__min_samples_leaf': 2}
